In [31]:
import pandas as pd
import json
import requests
from urllib import parse
# from elasticsearch import Elasticsearch
from os import name
import xml.etree.ElementTree as et
import bs4
from lxml import html
# from urllib.parse import urlencode, quote_plus, unquote
import xmltodict
import json

##  https://www.data.go.kr/data/15077644/openapi.do
소방청_화재정보서비스 -> 
소방청 국가화재정보 시도별화재인명피해현황 목록조회

In [30]:
api_name = ['getOcBysidoFireSmrzPcnd','getOcByfrstFireSmrzPcnd','getOcBysidoFpcnd','getOcBysidoFireOcrnPcnd','getOcFirePcnd',
            'getOcFirePrcnd','getOcArFireByplceFpcnd','getOcIgntnByfctrFpcnd','getOcIgntnBypntFpcnd','getOcIgntnByplceFpcnd',
            'getOcFrstBychlrdFpcnd','getOcStrcStrcstFpcnd','getOcRockByminfctyFpcnd','getOcVhclIgntnBypntFpcnd','getOcShipByarplFpcnd',
            'getOcBywdldFpcnd','getOcIgntnByahsFpcnd','getArFireByplceFpcnd','getOcRockMnfctyPcnd']
url_base = 'http://apis.data.go.kr/1661000/FireInformationService/'
key = 'WwrHmjfnt6hWvSuw%2FI%2FGOBndFtHW9C9MlIGXnN2O3LEhlG%2B2ZtVqjYXef9TiqpLftYFOzgag5A2NOAF1d4hx7A%3D%3D'

# 날짜는 2021-01-01 부터 ~ 2022-11-30

In [39]:
# 1~18
def api_func_1(num, date):

    queryParams = f'?{parse.quote_plus("serviceKey")}={key}&' + parse.urlencode({
    parse.quote_plus("pageNo"): '1',
    parse.quote_plus("numOfRows"):'1000',
    parse.quote_plus("resultType"): 'json',
    parse.quote_plus("ocrn_ymd"): date
    })

    url = url_base+api_name[num-1]
    response = requests.get(url +  queryParams).text
    json_ob = json.loads(response)
    body = json_ob['response']['body']['items']['item']
    dataframe = pd.json_normalize(body)    
    
    if num == 1: 
        df = dataframe.rename(
            columns={
                'sidoNm':'시도명', 
                'flsrpPrcsMnb':'오보처리건수', 
                'slfExtshMnb':'자체진화건수',
                'fireRcptMnb':'화재접수건수', 
                'stnEndMnb':'상황종료건수', 
                'ocrnYmd':'발생일자', 
                'falsDclrMnb':'허위신고건수'
                })
    elif num == 3:
        df = dataframe.rename(
            columns={
                'vctmPercnt':'사고자인원수', 
                'lifeDmgPercnt':'인명피해인원수', 
                'injrdprPercnt':'부상자인원수', 
                'sidoNm':'시도명',
                'ocrnMnb':'발생건수', 
                'ocrnYmd':'발생일자'
                })  

    return df

In [42]:
start_date='20210101'
end_date='20230213'
date_list=pd.date_range(start=start_date, end=end_date, freq='D')
date_list = date_list.astype('str')
date_list=date_list.str.replace(pat=r'[^\w]',repl=r'',regex=True)
date_list=date_list.tolist()

In [46]:
n = 3

for i in date_list:
    if i == date_list[0]:
        box = api_func_1(n,i)
    else:
        box = pd.concat([box,api_func_1(n,i)])

box.info()
# box.to_csv('./box'+n+'.csv',mode = 'w', index=False)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12183 entries, 0 to 13
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   사고자인원수   12183 non-null  int64 
 1   인명피해인원수  12183 non-null  int64 
 2   부상자인원수   12183 non-null  int64 
 3   시도명      12183 non-null  object
 4   발생건수     12183 non-null  int64 
 5   발생일자     12183 non-null  object
dtypes: int64(4), object(2)
memory usage: 666.3+ KB


In [49]:
box.to_csv('./box3.csv',mode = 'w', index=False)

In [55]:
box = box[['시도명','발생일자','발생건수','인명피해인원수']]
box.columns = ['REGION_NAME','DATE','FIRE_NUM','DEAD_NUM']
box

,REGION_NAME,DATE,FIRE_NUM,DEAD_NUM
0,서울특별시,20210101,14,2
1,부산광역시,20210101,2,0
2,대구광역시,20210101,1,0
3,인천광역시,20210101,6,0
4,광주광역시,20210101,5,1
...,...,...,...,...
9,충청남도,20230213,1,0
10,전라북도,20230213,4,0
11,전라남도,20230213,1,0
12,경상북도,20230213,4,0


In [56]:
box.to_csv('./FIRE.csv',mode = 'w', index=False)

## 읍면동 주민등록 인구

# !pip install PublicDataReader --upgrade

In [58]:
from PublicDataReader import Kosis

# KOSIS 공유서비스 Open API 사용자 인증키
service_key = "OTY3NjI0YWVkN2Q3ZjIxOTk4MGZiZDcwODMwZjVhYjY="

# 인스턴스 생성하기
api = Kosis(service_key)

## 읍면동 주민등록 인구

In [59]:
df = api.get_data(
    "KOSIS통합검색",
    searchNm="읍면동 주민등록 인구"
)
df.head(1)

,기관ID,기관명,통계표ID,통계표명,조사ID,조사명,KOSIS목록구분,KOSIS통계표위치,통계표위치,통계표주요내용,수록기간시작일,수록기간종료일,통계표주석,추천통계표여부,KOSIS목록URL,KOSIS통계표URL,검색결과건수,검색어명
0,101,행정안전부,DT_1B04005N,행정구역(읍면동)별/5세별 주민등록인구(2011년~),2008001,주민등록인구현황,MT_ZTITLE,인구 > 주민등록인구현황,A > A_7,행정구역(동읍면)별 5세별 총인구수 남자인구수 여자인구수 전국 서울특별시 종로구 청...,2011,2023,"* 등록구분의 ＂전체＂는 ＂거주자＂, ＂거주불명자＂, ＂재외국민＂이 포함된 자료입니...",N,https://kosis.kr/statisticsList/statisticsList...,http://kosis.kr/statHtml/statHtml.do?orgId=101...,610,읍면동 주민등록 인구


In [60]:
df = api.get_data(
    "통계표설명",
    "자료갱신일",
    orgId="101",
    tblId="DT_1B04005N"
)

df.groupby(by=['수록주기']).agg({"수록시점": ["min", "max"]})

수록시점        
         min     max
수록주기                
년       2011    2022
월     201207  202301

In [61]:
item = api.get_data(
    "통계표설명",
    "분류항목",
    orgId="101",
    tblId="DT_1B04005N",
)
item.head()

,기관ID,통계표ID,코드ID,코드명,분류ID,분류명,분류영문명,분류값ID,분류값명,분류값영문명,상위분류값ID,분류값순번,CD_ENG_NM
0,101,DT_1B04005N,14STD04553,명,ITEM,항목,Item code list,T2,총인구수,Population,NaN,NaN,Person
1,101,DT_1B04005N,14STD04553,명,ITEM,항목,Item code list,T3,남자인구수,Male,NaN,NaN,Person
2,101,DT_1B04005N,14STD04553,명,ITEM,항목,Item code list,T4,여자인구수,Female,NaN,NaN,Person
3,101,DT_1B04005N,NaN,NaN,A,행정구역(동읍면)별,By Administrative District,00,전국,Whole country,NaN,1,NaN
4,101,DT_1B04005N,NaN,NaN,A,행정구역(동읍면)별,By Administrative District,11,서울특별시,Seoul,NaN,1,NaN


In [62]:
item.loc[item["분류ID"]=="ITEM"]

,기관ID,통계표ID,코드ID,코드명,분류ID,분류명,분류영문명,분류값ID,분류값명,분류값영문명,상위분류값ID,분류값순번,CD_ENG_NM
0,101,DT_1B04005N,14STD04553,명,ITEM,항목,Item code list,T2,총인구수,Population,NaN,NaN,Person
1,101,DT_1B04005N,14STD04553,명,ITEM,항목,Item code list,T3,남자인구수,Male,NaN,NaN,Person
2,101,DT_1B04005N,14STD04553,명,ITEM,항목,Item code list,T4,여자인구수,Female,NaN,NaN,Person


In [63]:
item.loc[item["분류ID"]=="A"]

,기관ID,통계표ID,코드ID,코드명,분류ID,분류명,분류영문명,분류값ID,분류값명,분류값영문명,상위분류값ID,분류값순번,CD_ENG_NM
3,101,DT_1B04005N,NaN,NaN,A,행정구역(동읍면)별,By Administrative District,00,전국,Whole country,NaN,1,NaN
4,101,DT_1B04005N,NaN,NaN,A,행정구역(동읍면)별,By Administrative District,11,서울특별시,Seoul,NaN,1,NaN
5,101,DT_1B04005N,NaN,NaN,A,행정구역(동읍면)별,By Administrative District,11110,종로구,Jongno-gu,11,1,NaN
6,101,DT_1B04005N,NaN,NaN,A,행정구역(동읍면)별,By Administrative District,1111051500,청운효자동,Cheongunhyoja-dong,11110,1,NaN
7,101,DT_1B04005N,NaN,NaN,A,행정구역(동읍면)별,By Administrative District,1111053000,사직동,Sajik-dong,11110,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4187,101,DT_1B04005N,NaN,NaN,A,행정구역(동읍면)별,By Administrative District,5013058000,서홍동,Seohong-dong,50130,1,NaN
4188,101,DT_1B04005N,NaN,NaN,A,행정구역(동읍면)별,By Administrative District,5013059000,대륜동,Daeryun-dong,50130,1,NaN
4189,101,DT_1B04005N,NaN,NaN,A,행정구역(동읍면)별,By Administrative District,5013060000,대천동,Daecheon-dong,50130,1,NaN
4190,101,DT_1B04005N,NaN,NaN,A,행정구역(동읍면)별,By Administrative District,5013061000,중문동,Jungmun-dong,50130,1,NaN


In [64]:
item.loc[item["분류ID"]=="B"]

,기관ID,통계표ID,코드ID,코드명,분류ID,분류명,분류영문명,분류값ID,분류값명,분류값영문명,상위분류값ID,분류값순번,CD_ENG_NM
4192,101,DT_1B04005N,NaN,NaN,B,5세별,By Age Group (Five-Year),0,계,Total,NaN,2,NaN
4193,101,DT_1B04005N,NaN,NaN,B,5세별,By Age Group (Five-Year),5,0 - 4세,0-4 Years old,NaN,2,NaN
4194,101,DT_1B04005N,NaN,NaN,B,5세별,By Age Group (Five-Year),10,5 - 9세,5-9 Years old,NaN,2,NaN
4195,101,DT_1B04005N,NaN,NaN,B,5세별,By Age Group (Five-Year),15,10 - 14세,10-14 Years old,NaN,2,NaN
4196,101,DT_1B04005N,NaN,NaN,B,5세별,By Age Group (Five-Year),20,15 - 19세,15-19 Years old,NaN,2,NaN
4197,101,DT_1B04005N,NaN,NaN,B,5세별,By Age Group (Five-Year),25,20 - 24세,20-24 Years old,NaN,2,NaN
4198,101,DT_1B04005N,NaN,NaN,B,5세별,By Age Group (Five-Year),30,25 - 29세,25-29 Years old,NaN,2,NaN
4199,101,DT_1B04005N,NaN,NaN,B,5세별,By Age Group (Five-Year),35,30 - 34세,30-34 Years old,NaN,2,NaN
4200,101,DT_1B04005N,NaN,NaN,B,5세별,By Age Group (Five-Year),40,35 - 39세,35-39 Years old,NaN,2,NaN
4201,101,DT_1B04005N,NaN,NaN,B,5세별,By Age Group (Five-Year),45,40 - 44세,40-44 Years old,NaN,2,NaN


In [72]:
df = api.get_data(
    "통계자료",
    orgId="101",
    tblId="DT_1B04005N",
    objL1="00 11 26 27 28 29 30 31 36 41 42 43 44 45 46 47 48 50",
    objL2="0",
    itmId="T2",
    prdSe="M",
    startPrdDe="202001",
    endPrdDe="202301",
)
df = df[df.분류값명1 != '전국']

In [66]:
df.to_csv("시도별인구현황.csv")

In [73]:
pv = df.pivot(index=["분류값ID1","분류값명1","수록시점"], columns=["항목명"], values="수치값").reset_index()
pv.columns.name = None
pv['수록시점'] = pd.to_datetime(pv['수록시점'], format="%Y%m")
numCols = ["총인구수"]
for col in numCols:
    pv[col] = pd.to_numeric(pv[col])
pv

,분류값ID1,분류값명1,수록시점,총인구수
0,11,서울특별시,2020-01-01,9733509
1,11,서울특별시,2020-02-01,9736962
2,11,서울특별시,2020-03-01,9733655
3,11,서울특별시,2020-04-01,9726787
4,11,서울특별시,2020-05-01,9724496
...,...,...,...,...
624,50,제주특별자치도,2022-09-01,678426
625,50,제주특별자치도,2022-10-01,678324
626,50,제주특별자치도,2022-11-01,678373
627,50,제주특별자치도,2022-12-01,678159


In [77]:
pv = pv[['분류값명1','수록시점','총인구수']]
pv = pv.rename(columns={
                '분류값명1':'REGION_NAME', 
                '총인구수':'POPULATION'
                })

In [83]:
import datetime
pv['YEAR'] = pv['수록시점'].dt.year 
pv['MONTH'] = pv['수록시점'].dt.month 
pv = pv.drop(labels='수록시점',axis=1)
pv = pv[['REGION_NAME','YEAR','MONTH','POPULATION']]

In [84]:
pv.to_csv('./POPULATION.csv',mode = 'w', index=False)

# 창고업 운수업

In [58]:
df = api.get_data(
    "KOSIS통합검색",
    searchNm="시도별/산업별 총괄"
)
df

,기관ID,기관명,통계표ID,통계표명,조사ID,조사명,KOSIS목록구분,KOSIS통계표위치,통계표위치,통계표주요내용,수록기간시작일,수록기간종료일,통계표주석,추천통계표여부,KOSIS목록URL,KOSIS통계표URL,검색결과건수,검색어명
0,101,통계청,DT_2KB9001,시도/산업별 총괄,1988019,서비스업조사,MT_ZTITLE,도소매ㆍ서비스 > 서비스업조사 > 등록기반 > 10차 산업분류 개정(2020~),O > O_8 > O_8_001 > O_8_001_001,산업별 시도별 종사자수 남 기타경비 임차료 여 매출액 인건비 영업이익 영업비용 사업...,2020,2020,o 산업분류는 한국표준산업분류 제10차 개정(2017년 7월) 적용 o 2020년 ...,N,https://kosis.kr/statisticsList/statisticsList...,http://kosis.kr/statHtml/statHtml.do?orgId=101...,220,시도별/산업별 총괄
1,101,통계청,DT_2KI2065,시도/산업별 총괄,2010012,경제총조사,MT_ZTITLE,경제일반ㆍ경기 > 경제총조사 > 등록기반 > 10차 개정 > 2020년 > 특성편 ...,J1 > J1_3 > J1_3_001 > J1_3_001_001 > J1_3_001...,산업별 시도별 종사자수 기타경비 남 매출액 영업비용 사업체수 급여총액 여 매출원가 ...,2020,2020,o 산업분류는 한국표준산업분류 제10차 개정(2017년 7월) 적용 o 2020년 ...,N,https://kosis.kr/statisticsList/statisticsList...,http://kosis.kr/statHtml/statHtml.do?orgId=101...,220,시도별/산업별 총괄
2,101,통계청,DT_1I202001,시도별/산업별 총괄,1978009,운수업조사,MT_ZTITLE,"교통ㆍ물류 > 운수업조사 > 운수업조사(2020~: 10차 산업분류기준, SBR기준)",M2 > M2_13 > M2_13_001,산업별 시도별 매출액 유형자산 기업체수 영업비용 부가가치 종사자수 급여액 운송수단 ...,2019,2020,* 이용자 안내 2020년 기준 운수업조사 결과는 행정자료에 기초한 모집단을 새롭게...,N,https://kosis.kr/statisticsList/statisticsList...,http://kosis.kr/statHtml/statHtml.do?orgId=101...,220,시도별/산업별 총괄
3,101,통계청,DT_1KB9001,시도/산업별 총괄,1988019,서비스업조사,MT_ZTITLE,도소매ㆍ서비스 > 서비스업조사 > 조사기반 > 10차_서비스업조사(2016~),O > O_8 > O_8_002 > J_20_6,산업별 시도별 임차료 상대표준오차 매출원가 판매비와 관리비 기타경비 사업체수 종사자...,2016,2019,o 한국산업표준분류 10차개정 ＇17년 7월 시행 o 2016년부터 서비스업조사(도...,N,https://kosis.kr/statisticsList/statisticsList...,http://kosis.kr/statHtml/statHtml.do?orgId=101...,220,시도별/산업별 총괄
4,101,통계청,DT_1I100001,시도별/산업별 총괄,1978009,운수업조사,MT_ZTITLE,교통ㆍ물류 > 운수업조사 > 운수업조사(2007~2019: 10차 산업분류기준),M2 > M2_13 > I11_I10,산업별 시도별 운송수단 및 창고수 기업체수 급여액 영업비용 부가가치 유형자산 매출액...,2007,2019,* 특수여객자동차운송업 : 장의차량을 운영하는 산업활동 * 늘찬배달업 : 도시내에서...,N,https://kosis.kr/statisticsList/statisticsList...,http://kosis.kr/statHtml/statHtml.do?orgId=101...,220,시도별/산업별 총괄
5,101,통계청,DT_1KB6001,시도/산업별 총괄,1988019,서비스업조사,MT_ZTITLE,도소매ㆍ서비스 > 서비스업조사 > 조사기반 > 9차_서비스업조사(2007~2016),O > O_8 > O_8_002 > J_20_1,산업별 시도별 연간급여액 인건비 기타경비 매출액 영업비용 임차료 건물연면적 사업체수...,2006,2016,"o 2016년 기준 서비스업조사 통계자료(대분류 M. 전문과학기술서비스업, 9차산업...",N,https://kosis.kr/statisticsList/statisticsList...,http://kosis.kr/statHtml/statHtml.do?orgId=101...,220,시도별/산업별 총괄
6,101,통계청,DT_2KI2009,시・도별/산업세분류별 총괄,2010012,경제총조사,MT_ZTITLE,경제일반ㆍ경기 > 경제총조사 > 등록기반 > 10차 개정 > 2020년 > 지역편,J1 > J1_3 > J1_3_001 > J1_3_001_001 > J1_3_001...,산업분류(세분류) 행정구역(시도) 급여총액 경상연구개발비 사업체수 임차료 여 영업비...,2020,2020,,N,https://kosis.kr/statisticsList/statisticsList...,http://kosis.kr/statHtml/statHtml.do?orgId=101...,220,시도별/산업별 총괄
7,101,통계청,DT_2KI2016,시 도별/산업소분류별/조직형태별/본 지점별 총괄,2010012,경제총조사,MT_ZTITLE,경제일반ㆍ경기 > 경제총조사 > 등록기반 > 10차 개정 > 2020년 > 지역편,J1 > J1_3 > J1_3_001 > J1_3_001_001 > J1_3_001...,산업분류(소분류) 행정구역(시도) 조직형태 본 지점 사업체수 영업비용 종사자수 매출...,2020,2020,,N,https://kosis.kr/statisticsList/statisticsList...,http://kosis.kr/statHtml/statHtml.do?orgId=101...,220,시도별/산업별 총괄
8,101,통계청,DT_2KI2013,시 도별/산업소분류별/종사자규모별 총괄,2010012,경제총조사,MT_ZTITLE,경제일반ㆍ경기 > 경제총조사 > 등록기반 > 10차 개정 > 2020년 > 지역편,J1 > J1_3 > J1_3_001 > J1_3_001_001 > J1_3_001...,산업분류(소분류) 행정구역(시도) 종사자규모 인건비 급여총액 경상연구개발비 영업이익...,2020,2020,,N,https://kosis.kr/statisticsList/statisticsList...,http://kosis.kr/statHtml/statHtml.do?orgId=101...,220,시도별/산업별 총괄
9,101,통계청,DT_2KI2015,시 도별/산업소분류별/매출액규모별 총괄,2010012,경제총조사,MT_ZTITLE,경제일반ㆍ경기 > 경제총조사 > 등록기반 > 10차 개정 > 2020년 > 지역편,J1 > J1_3 > J1_3_001 > J1_3_001_001 > J1_3_001...,산업분류(소분류) 행정구역(시도) 매출액규모 종사자수 인건비 임차료 영업이익 남 매...,2020,2020,,N,https://kosis.kr/statisticsList/statisticsList...,http://kosis.kr/statHtml/statHtml.do?orgId=101...,220,시도별/산업별 총괄


In [59]:
df = api.get_data(
    "통계표설명",
    "자료갱신일",
    orgId="101",
    tblId="DT_1I202001"
)

df.groupby(by=['수록주기']).agg({"수록시점": ["min", "max"]})

수록시점      
       min   max
수록주기            
년     2019  2020

In [60]:
item = api.get_data(
    "통계표설명",
    "분류항목",
    orgId="101",
    tblId="DT_1I202001",
)
item

,기관ID,통계표ID,코드ID,코드명,분류ID,분류명,분류영문명,분류값ID,분류값명,분류값영문명,상위분류값ID,분류값순번,CD_ENG_NM
0,101,DT_1I202001,14STD03222,개,ITEM,항목,Item code list,T1,기업체수,Number of Enterprises,NaN,NaN,Establishment
1,101,DT_1I202001,14STD04552,명,ITEM,항목,Item code list,T2,종사자수,no. of workers,NaN,NaN,PERSONS
2,101,DT_1I202001,14STD05057,백만원,ITEM,항목,Item code list,T3,급여액,Remuneration,NaN,NaN,Million Won
3,101,DT_1I202001,14STD05329,수,ITEM,항목,Item code list,T4,운송수단 및 창고수,no. of Equipments,NaN,NaN,number
4,101,DT_1I202001,14STD05057,백만원,ITEM,항목,Item code list,T5,매출액,Revenue,NaN,NaN,Million Won
...,...,...,...,...,...,...,...,...,...,...,...,...,...
73,101,DT_1I202001,NaN,NaN,IND_10,산업별,By industry,52942,수상 화물 취급업,Marine cargo handling,52,2,NaN
74,101,DT_1I202001,NaN,NaN,IND_10,산업별,By industry,52991,통관 대리 및 관련서비스업,Activities of customs agents and related services,52,2,NaN
75,101,DT_1I202001,NaN,NaN,IND_10,산업별,By industry,52992,화물운송 중개 대리 및 관련 서비스업,Activities of freight forwarders and cargo age...,52,2,NaN
76,101,DT_1I202001,NaN,NaN,IND_10,산업별,By industry,52993,화물 포장 검수 및 계량 서비스업,Activities of cargo packing checking and weigh...,52,2,NaN


In [61]:
item['분류ID'].unique().tolist()

['ITEM', 'SGG', 'IND_10']

In [62]:
item.loc[item["분류ID"]=="ITEM"]

,기관ID,통계표ID,코드ID,코드명,분류ID,분류명,분류영문명,분류값ID,분류값명,분류값영문명,상위분류값ID,분류값순번,CD_ENG_NM
0,101,DT_1I202001,14STD03222,개,ITEM,항목,Item code list,T1,기업체수,Number of Enterprises,NaN,NaN,Establishment
1,101,DT_1I202001,14STD04552,명,ITEM,항목,Item code list,T2,종사자수,no. of workers,NaN,NaN,PERSONS
2,101,DT_1I202001,14STD05057,백만원,ITEM,항목,Item code list,T3,급여액,Remuneration,NaN,NaN,Million Won
3,101,DT_1I202001,14STD05329,수,ITEM,항목,Item code list,T4,운송수단 및 창고수,no. of Equipments,NaN,NaN,number
4,101,DT_1I202001,14STD05057,백만원,ITEM,항목,Item code list,T5,매출액,Revenue,NaN,NaN,Million Won
5,101,DT_1I202001,14STD05057,백만원,ITEM,항목,Item code list,T6,영업비용,Operating expences,NaN,NaN,Million Won
6,101,DT_1I202001,14STD05057,백만원,ITEM,항목,Item code list,T7,부가가치,Value added,NaN,NaN,Million Won
7,101,DT_1I202001,14STD05057,백만원,ITEM,항목,Item code list,T8,유형자산,Fixed assets,NaN,NaN,Million Won


In [63]:
item.loc[item["분류ID"]=="SGG"]

,기관ID,통계표ID,코드ID,코드명,분류ID,분류명,분류영문명,분류값ID,분류값명,분류값영문명,상위분류값ID,분류값순번,CD_ENG_NM
8,101,DT_1I202001,NaN,NaN,SGG,시도별,By province,00,전국,Whole country,NaN,1,NaN
9,101,DT_1I202001,NaN,NaN,SGG,시도별,By province,11,서울특별시,Seoul,NaN,1,NaN
10,101,DT_1I202001,NaN,NaN,SGG,시도별,By province,21,부산광역시,Busan,NaN,1,NaN
11,101,DT_1I202001,NaN,NaN,SGG,시도별,By province,22,대구광역시,Daegu,NaN,1,NaN
12,101,DT_1I202001,NaN,NaN,SGG,시도별,By province,23,인천광역시,Incheon,NaN,1,NaN
13,101,DT_1I202001,NaN,NaN,SGG,시도별,By province,24,광주광역시,Gwangju,NaN,1,NaN
14,101,DT_1I202001,NaN,NaN,SGG,시도별,By province,25,대전광역시,Daejeon,NaN,1,NaN
15,101,DT_1I202001,NaN,NaN,SGG,시도별,By province,26,울산광역시,Ulsan,NaN,1,NaN
16,101,DT_1I202001,NaN,NaN,SGG,시도별,By province,29,세종특별자치시,Sejong,NaN,1,NaN
17,101,DT_1I202001,NaN,NaN,SGG,시도별,By province,31,경기도,Gyeonggi-do,NaN,1,NaN


In [64]:
item.loc[item["분류ID"]=="IND_10"]

,기관ID,통계표ID,코드ID,코드명,분류ID,분류명,분류영문명,분류값ID,분류값명,분류값영문명,상위분류값ID,분류값순번,CD_ENG_NM
26,101,DT_1I202001,NaN,NaN,IND_10,산업별,By industry,00,총계,Total,NaN,2,NaN
27,101,DT_1I202001,NaN,NaN,IND_10,산업별,By industry,49,육상운송 및 파이프라인 운송업,Land transport and transport via pipelines,NaN,2,NaN
28,101,DT_1I202001,NaN,NaN,IND_10,산업별,By industry,49101,철도 여객 운송업,Passenger rail transport interurban,49,2,NaN
29,101,DT_1I202001,NaN,NaN,IND_10,산업별,By industry,49102,철도 화물 운송업,Freight rail transport,49,2,NaN
30,101,DT_1I202001,NaN,NaN,IND_10,산업별,By industry,49211,도시철도 운송업,Urban railway transport,49,2,NaN
31,101,DT_1I202001,NaN,NaN,IND_10,산업별,By industry,49212,시내버스 운송업,Intra-city bus transport,49,2,NaN
32,101,DT_1I202001,NaN,NaN,IND_10,산업별,By industry,49219,기타 도시 정기 육상 여객 운송업,Other urban and suburban passenger land transport,49,2,NaN
33,101,DT_1I202001,NaN,NaN,IND_10,산업별,By industry,49220,시외버스 운송업,Intercity bus transport,49,2,NaN
34,101,DT_1I202001,NaN,NaN,IND_10,산업별,By industry,49231,택시 운송업,Taxi transport,49,2,NaN
35,101,DT_1I202001,NaN,NaN,IND_10,산업별,By industry,49232,전세버스 운송업,Chartered bus transport,49,2,NaN


In [65]:
df = api.get_data(
    "통계자료",
    orgId="101",
    tblId="DT_1I202001",
    objL1="ALL",
    objL2="00 52",
    itmId="T1",
    prdSe="Y",
    startPrdDe="2020",
    endPrdDe="2020",
)
df

,기관ID,통계표ID,통계표명,분류명1,분류영문명1,분류값명1,분류값영문명1,분류값ID1,분류명2,분류영문명2,...,분류값영문명2,분류값ID2,항목ID,항목명,항목영문명,단위명,단위영문명,수록주기,수록시점,수치값
0,101,DT_1I202001,시도별/산업별 총괄,시도별,By province,전국,Whole country,00,산업별,By industry,...,Total,00,T1,기업체수,Number of Enterprises,개,Establishment,A,2020,552075
1,101,DT_1I202001,시도별/산업별 총괄,시도별,By province,전국,Whole country,00,산업별,By industry,...,Warehousing and support activities for transpo...,52,T1,기업체수,Number of Enterprises,개,Establishment,A,2020,22471
2,101,DT_1I202001,시도별/산업별 총괄,시도별,By province,서울특별시,Seoul,11,산업별,By industry,...,Total,00,T1,기업체수,Number of Enterprises,개,Establishment,A,2020,100240
3,101,DT_1I202001,시도별/산업별 총괄,시도별,By province,서울특별시,Seoul,11,산업별,By industry,...,Warehousing and support activities for transpo...,52,T1,기업체수,Number of Enterprises,개,Establishment,A,2020,5057
4,101,DT_1I202001,시도별/산업별 총괄,시도별,By province,부산광역시,Busan,21,산업별,By industry,...,Total,00,T1,기업체수,Number of Enterprises,개,Establishment,A,2020,31057
5,101,DT_1I202001,시도별/산업별 총괄,시도별,By province,부산광역시,Busan,21,산업별,By industry,...,Warehousing and support activities for transpo...,52,T1,기업체수,Number of Enterprises,개,Establishment,A,2020,3957
6,101,DT_1I202001,시도별/산업별 총괄,시도별,By province,대구광역시,Daegu,22,산업별,By industry,...,Total,00,T1,기업체수,Number of Enterprises,개,Establishment,A,2020,23576
7,101,DT_1I202001,시도별/산업별 총괄,시도별,By province,대구광역시,Daegu,22,산업별,By industry,...,Warehousing and support activities for transpo...,52,T1,기업체수,Number of Enterprises,개,Establishment,A,2020,877
8,101,DT_1I202001,시도별/산업별 총괄,시도별,By province,인천광역시,Incheon,23,산업별,By industry,...,Total,00,T1,기업체수,Number of Enterprises,개,Establishment,A,2020,26506
9,101,DT_1I202001,시도별/산업별 총괄,시도별,By province,인천광역시,Incheon,23,산업별,By industry,...,Warehousing and support activities for transpo...,52,T1,기업체수,Number of Enterprises,개,Establishment,A,2020,1609


In [4]:
url = 'https://infuser.odcloud.kr/oas/docs?namespace=15063546/v1/uddi:cff5c093-4893-454f-96af-f2764590defd'
key = 'buNPq8BZdGw2p%2B3l2B7IMBy3e9PX5fahBKLPiW518y4yMZsQFDJeQW9fpNMb2nt7Xu6j4vz6txHwT5%2BrcFrwhA%3D%3D'


queryParams = f'?{parse.quote_plus("serviceKey")}={key}&' + parse.urlencode({
#parse.quote_plus("ocrn_ymd"): '20230119',
#parse.quote_plus("siDo"): '1100',
#parse.quote_plus("resultType"): 'xml' ,
#parse.quote_plus("numOfRows"): 100,
#parse.quote_plus("pageNo"): 1
})

# xml로 받아온 response
response = requests.get(url + queryParams).text
# xmltodict로 dictionary형태로 바꿔줌
jsonString = json.dumps(xmltodict.parse(response), indent=4)
# dictiondary to json
dic = json.loads(jsonString)
dic
#body = dic['response']['body']['items']['item']

#df1 = pd.json_normalize(body)

ExpatError: not well-formed (invalid token): line 1, column 0